# ImgProcess

Get the GPS coordinates and sync the images

In [1]:
from exif import Image
from datetime import datetime
import re
import pickle

In [2]:
files = !ls img_walnutst/*.jpg
files

['img_walnutst/IMG_20230914_114712382_HDR.jpg',
 'img_walnutst/IMG_20230914_114815069_HDR.jpg',
 'img_walnutst/IMG_20230914_114839959_HDR.jpg',
 'img_walnutst/IMG_20230914_114854052_HDR.jpg',
 'img_walnutst/IMG_20230914_114916496.jpg',
 'img_walnutst/IMG_20230914_115003807_HDR.jpg',
 'img_walnutst/IMG_20230914_115054516_HDR.jpg',
 'img_walnutst/IMG_20230914_115314080.jpg',
 'img_walnutst/IMG_20230914_115323231_HDR.jpg',
 'img_walnutst/IMG_20230914_115336872_HDR.jpg',
 'img_walnutst/IMG_20230914_115418289_HDR.jpg',
 'img_walnutst/IMG_20230914_115531381.jpg',
 'img_walnutst/IMG_20230914_115708259_HDR.jpg',
 'img_walnutst/IMG_20230914_115813496_HDR.jpg',
 'img_walnutst/IMG_20230914_115829542_HDR.jpg',
 'img_walnutst/IMG_20230914_121932518.jpg']

In [3]:
s3Loc = "https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/"

In [4]:
def convertGPS(cord):
    outCord = cord[0]
    
    outCord += cord[1] / 60.0
    
    outCord += cord[2] / 3600.0
    
    return outCord

In [11]:
def getMetaData(files, reg = re.compile('^img_walnutst/IMG_([^_]+)'), s3Loc = s3Loc):

    outList = []
    for file in files:
        with open(file, 'rb') as fileObj:
            img = Image(fileObj)

            lat = convertGPS( img.gps_latitude )
            lon = convertGPS( img.gps_longitude ) * -1.0

            try:
                timeStr = reg.match(file).groups()[0]
            except Exception as e:
                print(file)
                raise(e)

            date = datetime.strptime(timeStr, '%Y%m%d')
            
            outList.append({
                        'url':s3Loc+file,
                      'file':file,
                      'coords':(lon,lat),
                      'date':date,
                      'originalLat':img.gps_latitude,
                      'originalLon': img.gps_longitude
                     })
            
            
            

    return outList
              
        
        


In [12]:
metaData = getMetaData(files) 
metaData

[{'url': 'https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/IMG_20230914_114712382_HDR.jpg',
  'file': 'img_walnutst/IMG_20230914_114712382_HDR.jpg',
  'coords': (-90.13142394444444, 29.927722916666667),
  'date': datetime.datetime(2023, 9, 14, 0, 0),
  'originalLat': (29.0, 55.0, 39.8025),
  'originalLon': (90.0, 7.0, 53.1262)},
 {'url': 'https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/IMG_20230914_114815069_HDR.jpg',
  'file': 'img_walnutst/IMG_20230914_114815069_HDR.jpg',
  'coords': (-90.13134, 29.92795180555556),
  'date': datetime.datetime(2023, 9, 14, 0, 0),
  'originalLat': (29.0, 55.0, 40.6265),
  'originalLon': (90.0, 7.0, 52.824)},
 {'url': 'https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/img_walnutst/IMG_20230914_114839959_HDR.jpg',
  'file': 'img_walnutst/IMG_20230914_114839959_HDR.jpg',
  'coords': (-90.13101194444444, 29.92823788888889),
  'date': datetime.datetime(2023, 9, 14, 0, 0),
  'originalLat': (29.0, 55.0, 41.

In [7]:
with open('img_walnutstMetaData.pickle', 'wb') as f:
    pickle.dump(metaData,f)

In [8]:
str(datetime.now())

'2023-09-15 09:38:01.867530'

!aws s3 sync --delete ./img_walnutst s3://filestogeaux.garyscorner.net/pub/img_walnutst/ --acl public-read

In [9]:
str(datetime.now())

'2023-09-15 09:38:01.880025'

In [10]:
'done!'

'done!'